In [13]:
import pandas as pd
import numpy as np
from scipy.stats import gamma
def calculate_spi(precipitation_data, current_week_data):
    # Fit gamma distribution to historical precipitation data
    shape, loc, scale = gamma.fit(precipitation_data)
    
    # Calculate CDF for the current week based on the fitted gamma distribution
    cdf_values_current_week = gamma.cdf(current_week_data, shape, loc, scale)
    
    # Calculate SPI values for the current week using the inverse of the CDF
    spi_values_current_week = gamma.ppf(cdf_values_current_week, shape, loc, scale)
    
    return spi_values_current_week
df = pd.read_csv('Kathmandu_precipitaiton_daily.csv')
df = df.iloc[4:]
df.columns = ['year','month','day','temperature']
df['year'] = df['year'].astype(int)
df['month'] = df['month'].astype(int)
df['day'] = df['day'].astype(int)
df['temperature'] = df['temperature'].astype(float)
df['date'] = pd.to_datetime(df[['year', 'month', 'day']].astype(int).astype(str).agg('-'.join, axis=1))
df=df.drop(['year','month','day'],axis=1)
df.set_index('date',inplace=True)
df.index = pd.to_datetime(df.index)

df['temperature'] = df['temperature'].replace(-9999,np.nan)
df = df[df.index.year != 2023]
weekly_df = pd.DataFrame()
for month_start, month_group in df.groupby(df.index.to_period("M")):
    # Iterate over each week within the month
    for week_start, week_group in month_group.resample('W').sum().iterrows():
        # Calculate the accumulated sum for the last 28 days, including data from the previous month
        start_date = week_start - pd.DateOffset(days=27)
        accumulated_sum = original_df['Precipitation'].loc[start_date:week_start].sum()

        # Create a new row for the week in the weekly DataFrame
        weekly_row = {
            'Month': month_start.strftime('%B'),  # Month name
            'Week_Start_Date': start_date,   # Start date of the week
            'Week_End_Date': week_start,    # End date of the week
            'Accumulated_Sum': accumulated_sum
        }

        # Concatenate the row to the weekly DataFrame
        weekly_df = pd.concat([weekly_df, pd.DataFrame([weekly_row])], ignore_index=True)
# Display the resulting DataFrame
print(weekly_df)

df = df.groupby(df.index.year)['temperature'].sum().reset_index()
print(df)
df.set_index('date',inplace=True)
df.index = pd.to_datetime(df.index)

current_year = df[df.index.year==2007]['temperature'].sum()
spi = calculate_spi(df['temperature'].values,df['temperature'].values)
array_in_decimal_values = np.float64(spi)
print(array_in_decimal_values)

NameError: name 'original_df' is not defined

In [12]:
import numpy as np
from scipy.stats import gamma
from scipy.signal import convolve

def calculate_spi_for_pixel(pixel_data, time_scale):
    # Convert pixel data to a numpy array
    pixel_array = np.array(pixel_data)

    # Calculate the gamma distribution parameters (fit to pixel data)
    shape, loc, scale = gamma.fit(pixel_array.flatten(), floc=0)

    # Calculate the gamma distribution cumulative distribution function (CDF)
    cdf = gamma.cdf(pixel_array, shape, loc, scale)

    # Calculate the expected precipitation for each time step
    expected_precip = np.mean(pixel_array, axis=0)

    # Calculate the SPI values for each time step
    spi_values = np.zeros_like(pixel_array)
    for t in range(pixel_array.shape[0]):
        spi_values[t, ...] = gamma.ppf(cdf[t, ...], shape, loc, scale) - expected_precip

    # Convolve the SPI values with a 1-1-1 filter to obtain accumulated SPI values
    spi_accumulated = convolve(spi_values, [1, 1, 1], mode='same', method='direct')

    return spi_accumulated

# Example usage:
# Replace this with your actual precipitation data (3D array)
historical_precip_data = np.random.rand(365, 10, 10)  # Example: 365 days, 10x10 grid

# Specify the desired time scale for SPI calculation (e.g., 1 month, 3 months, etc.)
time_scale = 3  # Replace with your desired time scale in months

# Initialize an empty array to store SPI results for each pixel
spi_results = np.zeros_like(historical_precip_data)

# Calculate SPI for each pixel
for i in range(historical_precip_data.shape[1]):  # Assuming the second dimension represents the rows
    for j in range(historical_precip_data.shape[2]):  # Assuming the third dimension represents the columns
        pixel_data = historical_precip_data[:, i, j]
        spi_results[:, i, j] = calculate_spi_for_pixel(pixel_data, time_scale)